In [1]:
# Step 1: Install Required Libraries
!pip install tensorflow tensorflow-hub tensorflow-datasets matplotlib numpy scikit-learn

In [2]:
# Step 2: Import Necessary Libraries
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score


In [3]:
import tensorflow_datasets as tfds

In [4]:
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: []


In [5]:
dataset_name = "voc/2007"
split_ratio = 0.2  # Use 20% of the dataset
batch_size = 32
image_size = (300, 300)

In [6]:
pip install --upgrade tensorflow-datasets

Note: you may need to restart the kernel to use updated packages.


In [7]:
import tensorflow_datasets as tfds

In [8]:
# Import necessary libraries
import tensorflow as tf
import tensorflow_datasets as tfds  # TensorFlow Datasets for dataset handling
import numpy as np
import matplotlib.pyplot as plt
import os

In [9]:
import tensorflow_datasets as tfds

In [ ]:
(train_dataset, test_dataset), info = tfds.load(
    dataset_name,
    split=["train[:80%]", "train[80%:]"],
    with_info=True,
    as_supervised=True
)
print("Dataset loaded. Classes:", info.features["labels"].names)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
def preprocess(image, labels):
    image = tf.image.resize(image, image_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image, labels

train_dataset = train_dataset.map(preprocess).batch(batch_size).prefetch(1)
test_dataset = test_dataset.map(preprocess).batch(batch_size).prefetch(1)


In [ ]:
def augment(image, labels):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.2)
    return image, labels

train_dataset = train_dataset.map(augment)

In [ ]:
for images, labels in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())
        plt.title(labels[i].numpy())
        plt.axis("off")
    plt.show()

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(300, 300, 3), include_top=False, weights="imagenet"
)

In [ ]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(info.features["labels"].names), activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
epochs = 10
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    verbose=1
)


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.legend()
plt.title("Training and Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")


In [ ]:
model.save("object_detection_model.h5")
print("Model saved as object_detection_model.h5")


In [ ]:
for images, labels in test_dataset.take(1):
    predictions = model.predict(images)
    predicted_labels = tf.argmax(predictions, axis=1)

In [ ]:
  plt.figure(figsize=(10, 10))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())
        plt.title(f"Predicted: {predicted_labels[i].numpy()}, True: {labels[i].numpy()}")
        plt.axis("off")
    plt.show()